# 1. Data Import

This notebook and enviroment is completely hosted on the cloud, and downloads the required files to your Google Drive.  To replicate this analysis:

1.   Make a copy of this Notebook to your Drive, and work in the new copy.
2.   Enter this competition https://www.kaggle.com/c/nyc-taxi-trip-duration/data by clicking on Late Submission.
3.   Download your Kaggle API token by going to My Account -> Create New API Token


In [1]:
! pip install -q kaggle

# Choose the kaggle.json file that you downloaded
from google.colab import files
files.upload()

# Make directory named kaggle and copy kaggle.json file there.
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

# Change the permissions of the file.
! chmod 600 ~/.kaggle/kaggle.json

# Check on data
! kaggle datasets list


Saving kaggle.json to kaggle.json
ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
kimjihoo/coronavirusdataset                                 Coronavirus-Dataset                                  32KB  2020-03-12 19:16:41          11195  
sudalairajkumar/novel-corona-virus-2019-dataset             Novel Corona Virus 2019 Dataset                     348KB  2020-03-12 10:57:12          37270  
paultimothymooney/coronavirus-genome-sequence               Coronavirus Genome Sequence                           9MB  2020-02-29 00:25:13            124  
imdevskp/sars-outbreak-2003-complete-dataset                SARS 2003 Outbreak Complete Dataset                  10KB  2020-02-26 10:25:22            982  
imdevskp/ebola-outbreak-201420

In [3]:
# download the Kaggle data to the remote machine
!kaggle competitions download -c nyc-taxi-trip-duration

 25% 5.00M/20.3M [00:00<00:00, 37.5MB/s]
100% 20.3M/20.3M [00:00<00:00, 80.8MB/s]
 70% 44.0M/62.9M [00:00<00:00, 69.7MB/s]
100% 62.9M/62.9M [00:00<00:00, 128MB/s] 
  0% 0.00/2.49M [00:00<?, ?B/s]
100% 2.49M/2.49M [00:00<00:00, 82.7MB/s]


In [15]:
import pandas as pd
import os
# check current working directory for file traversal
print(os.getcwd())

/content


In [10]:
#unzip the training files
!unzip -qq /content/train.zip -d /content/ 

replace /content/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [13]:
#unzip the test files
!unzip -qq /content/test.zip -d /content/ 

replace /content/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [0]:
df_taxi = pd.read_csv("train.csv")

In [17]:
df_taxi.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
